In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE
import sys 
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
from collections import Counter
from sklearn.metrics import  accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def generatebatch(X,Y,n_examples, batch_size): 
    for batch_i in range(n_examples // batch_size): 
        start = batch_i * batch_size 
        end = start + batch_size 
        batch_xs = X[start:end] 
        batch_ys = Y[start:end] 
        yield batch_xs, batch_ys 
    
data = pd.read_csv('E:\\备份\\训练数据汇总(8W)\\2019_4_15.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

name = ['PHONE_VERSION', 'VIDEO_CLARITY']
columns = data.columns.values.tolist()
name_1 = [name for index, name in enumerate(columns) if name != name[0] 
          and name != name[1]]

scaler = MinMaxScaler()
X1 = data[name_1[0:16]]
X1_data = scaler.fit_transform(X1)
Y = data[name_1[16:21]]
X2 = data[name[0]]
X3 = data[name[1]]

X2_data = OneHotEncoder().fit_transform(X2.values.reshape(-1, 1)).todense().getA()
X3_data = OneHotEncoder().fit_transform(X3.values.reshape(-1, 1)).todense().getA()

X_data = np.hstack((X1_data, X2_data, X3_data))

Y1 = data[[name_1[16]]]
Y2 = data[[name_1[17]]]
Y3 = data[[name_1[18]]]
Y4 = data[[name_1[19]]]

smo = SMOTE(random_state=42)
X1_smo, y1_smo = smo.fit_sample(X_data, Y1)

smo = SMOTE(random_state=42)
X2_smo, y2_smo = smo.fit_sample(X_data, Y2)

smo = SMOTE(random_state=42)
X3_smo, y3_smo = smo.fit_sample(X_data, Y3)

smo = SMOTE(random_state=42)
X4_smo, y4_smo = smo.fit_sample(X_data, Y4)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1_smo, y1_smo, 
                                                        test_size=0.25, 
                                                        random_state = 33)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_smo, y2_smo, 
                                                        test_size=0.25, 
                                                        random_state = 33)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3_smo, y3_smo, 
                                                        test_size=0.25, 
                                                        random_state = 33)
X4_train, X4_test, y4_train, y4_test = train_test_split(X4_smo, y4_smo, 
                                                        test_size=0.25, 
                                                        random_state = 33)
X1_train = X1_train.reshape(-1 ,1, 4, 7)
X1_test = X1_test.reshape(-1, 1 ,4,7 )
y1_train = OneHotEncoder().fit_transform(y1_train.reshape(-1, 1)).todense().getA()
y1_test_code = OneHotEncoder().fit_transform(y1_test.reshape(-1, 1)).todense().getA()

In [ ]:
# 使用MBGD算法，设定batch_size为8
batch_size = 1024
tf.reset_default_graph()

# 输入层
keep_prob = tf.placeholder(tf.float32)
tf_X = tf.placeholder(tf.float32,[None,1,4,7])
tf_Y = tf.placeholder(tf.float32,[None,5])

# 卷积层+激活层 
relu_feature_maps1 = tf.nn.relu(tf.layers.conv2d(tf_X,
                                                 kernel_size=[3,3],
                                                 strides=[1,1],
                                                 filters=5, 
                                                 padding = 'SAME'))
relu_feature_maps2 = tf.nn.relu(tf.layers.conv2d(relu_feature_maps1,
                                                 kernel_size=[2,2],
                                                 strides=[1,1],
                                                 filters=5, 
                                                 padding = 'SAME'))

max_pool2_flat = tf.reshape(relu_feature_maps2, [-1,5*1*4])
fc1 = tf.layers.dense(max_pool2_flat,units=50,activation=tf.nn.relu)

fc2 = tf.layers.dense(fc1,units=5,activation=tf.nn.relu)

pred = tf.nn.softmax(fc2)

loss = -1*tf.reduce_mean(tf_Y*tf.log(tf.clip_by_value(pred, 1e-11, 1.0)))
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

y_pred = tf.argmax(pred, 1)
bool_pred = tf.equal(tf.argmax(tf_Y,1), y_pred)
# 准确率
accuracy = tf.reduce_mean(tf.cast(bool_pred, tf.float32))

print("######### The train & test process of SCORE1 ##########")
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) 
    for epoch in range(1000): 
        # 迭代1000个周期 
        for batch_xs,batch_ys in generatebatch(X1_train, 
                                               y1_train, 
                                               y1_train.shape[0], 
                                               batch_size): 
            # 每个周期进行MBGD算法 
            sess.run(train_step, feed_dict={tf_X:batch_xs, 
                                            tf_Y:batch_ys, 
                                            keep_prob: 0.5}) 
        if(epoch%100==0): 
            res = sess.run(accuracy,feed_dict={tf_X:X1_train, 
                                               tf_Y:y1_train, 
                                               keep_prob: 0.5}) 
            #print((epoch,res))
            print("The %d epoch, the accuracy is %f " %(epoch, res))
    res_ypred = y_pred.eval(feed_dict={tf_X:X1_test, 
                                       tf_Y:y1_test_code, 
                                       keep_prob: 1}).flatten() 
    # 只能预测一批样本，不能预测一个样本 
    print('The result of the test bath is  ', res_ypred)
print("The accurate score of the test bath is  ", 
      accuracy_score(y1_test, res_ypred.reshape(-1, 1)))
print("The MSE of the test bach is  ",  
      mean_squared_error(y1_test, res_ypred.reshape(-1, 1)))

######### The train & test process of SCORE1 ##########
The 0 epoch, the accuracy is 0.341475 
The 100 epoch, the accuracy is 0.437777 
The 200 epoch, the accuracy is 0.453246 
The 300 epoch, the accuracy is 0.460920 
The 400 epoch, the accuracy is 0.466070 
The 500 epoch, the accuracy is 0.475116 
